In [9]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import os
import sys

# Fix import path (important for deployment)
sys.path.append(os.getcwd())

from app.agent.executor import Executor

app = FastAPI(
    title="Agentic AI System",
    description="API to run Agentic AI tasks",
    version="1.0"
)

agent = Executor()

class AgentRequest(BaseModel):
    goal: str
    input_file: str

@app.post("/run-agent")
def run_agent(request: AgentRequest):
    try:
        result = agent.execute(
            goal=request.goal,
            input_file=request.input_file
        )

        # Ensure the 'output' directory exists before writing
        if not os.path.exists('output'):
            os.makedirs('output')

        # For demonstration, let's write something to output.txt if it doesn't exist
        # In a real scenario, the agent.execute would handle file output
        if not os.path.exists("output/output.txt"):
            with open("output/output.txt", "w", encoding="utf-8") as f:
                f.write(f"Goal: {request.goal}, Input File: {request.input_file}\n")
                f.write(f"Agent Result (placeholder): {result}")

        with open("output/output.txt", "r", encoding="utf-8") as f:
            output_text = f.read()

        return {
            "status": "success",
            "message": result,
            "output": output_text
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [25]:
!pip install "uvicorn[standard]"

/usr/lib/python3.12/pathlib.py:404: RuntimeWarning: coroutine 'Server.serve' was never awaited
  parsed = [sys.intern(str(x)) for x in rel.split(sep) if x and x != '.']


Once the Uvicorn server is running, you will see output with a local address (e.g., `http://0.0.0.0:8000`). If you are running this in a Colab environment, you might need to use a tunneling service like ngrok to expose your local server to the internet. Colab often provides an ngrok URL automatically if you run the server this way. You can then access your API endpoints, for example, by navigating to `/docs` for the Swagger UI (`<ngrok_url>/docs`).

First, let's write your FastAPI application code into a `main.py` file. This is crucial for running Uvicorn as a separate process in the background, which is the most reliable way in Colab to avoid event loop conflicts.

In [26]:
app_content = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import os
import sys

# Fix import path (important for deployment)
sys.path.append(os.getcwd())

from app.agent.executor import Executor

app = FastAPI(
    title="Agentic AI System",
    description="API to run Agentic AI tasks",
    version="1.0"
)

agent = Executor()

class AgentRequest(BaseModel):
    goal: str
    input_file: str

@app.post("/run-agent")
def run_agent(request: AgentRequest):
    try:
        result = agent.execute(
            goal=request.goal,
            input_file=request.input_file
        )

        # Ensure the 'output' directory exists before writing
        if not os.path.exists('output'):
            os.makedirs('output')

        # For demonstration, let's write something to output.txt if it doesn't exist
        # In a real scenario, the agent.execute would handle file output
        if not os.path.exists("output/output.txt"):
            with open("output/output.txt", "w", encoding="utf-8") as f:
                f.write("Goal: {}, Input File: {}\n".format(request.goal, request.input_file))
                f.write("Agent Result (placeholder): {}\n".format(result))

        with open("output/output.txt", "r", encoding="utf-8") as f:
            output_text = f.read()

        return {
            "status": "success",
            "message": result,
            "output": output_text
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
'''

with open('main.py', 'w') as f:
    f.write(app_content)

print("Created main.py with your FastAPI application.")

Created main.py with your FastAPI application.


Now that `main.py` is created, you can run Uvicorn as a background process. This will start your FastAPI server. Colab typically provides a public URL (like an ngrok URL) that you can use to access your API endpoints, for example, by navigating to `/docs` for the Swagger UI (`<colab_url>/docs`).

In [18]:
# Start Uvicorn as a background process. The `&` symbol runs it in the background.
# `main:app` refers to the `app` object in the `main.py` file.
!uvicorn main:app --host 0.0.0.0 --port 8000 &

Traceback (most recent call last):
  File "/usr/local/bin/uvicorn", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1485, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1406, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 1269, in invoke
    return ctx.invoke(self.callback, **ctx.params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/click/core.py", line 824, in invoke
    return callback(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/main.py", line 423, in main
    run(
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/main.py", line 593, in run
    server.run()
  File "/usr/local/lib/python3.12/

In [19]:
import os

# Create the 'app' directory if it doesn't exist
if not os.path.exists('app'):
    os.makedirs('app')

# Create the 'app/agent' directory if it doesn't exist
if not os.path.exists('app/agent'):
    os.makedirs('app/agent')

print("Created 'app' and 'app/agent' directories.")

Created 'app' and 'app/agent' directories.


In [20]:
# Create __init__.py files to make 'app' and 'app.agent' Python packages
with open('app/__init__.py', 'w') as f:
    f.write('')

with open('app/agent/__init__.py', 'w') as f:
    f.write('')

print("Created '__init__.py' files.")

Created '__init__.py' files.


In [8]:
# Create a placeholder executor.py file with a basic Executor class
executor_content = """
class Executor:
    def execute(self, goal: str, input_file: str):
        print(f"Executing agent with goal: {goal} and input file: {input_file}")
        # Placeholder for actual agent logic
        return "Agent execution completed (placeholder)."
"""

with open('app/agent/executor.py', 'w') as f:
    f.write(executor_content)

print("Created 'app/agent/executor.py' with a placeholder Executor class.")

Created 'app/agent/executor.py' with a placeholder Executor class.


In [24]:
uvicorn app.api:app --reload



SyntaxError: invalid syntax (ipython-input-1160918112.py, line 1)